In [1]:
import numpy as np
import pandas as pd

flight_data = pd.read_csv('/Users/xuxinyi/Downloads/sort.csv', index_col=0) # remove the index column
flight_data.head()

,i1_legid,i1_hops,i1_dep_1_place,i1_rcf_1_place,i1_rcf_1_p,i1_rcf_1_e,i1_dep_2_place,i1_rcf_2_place,i1_rcf_2_p,i1_rcf_2_e,...,lead_time,value,cargo_price_unit,weight,risk_1,risk_2,risk_3,risk_all,distance,cargo_price
0,5878,1,DOH,PVG,905,547,?,?,0,0,...,547,90059,0.52,2.49,2.0,0.0,0.0,2.0,4220.0,5464.06
1,9889,1,FRA,LHR,215,223,?,?,0,0,...,223,3786,0.40,1.25,3.0,0.0,0.0,3.0,406.0,203.00
2,3997,1,FRA,ZRH,270,135,?,?,0,0,...,135,11143,0.62,0.36,3.5,0.0,0.0,3.5,177.0,39.51
3,10481,1,FRA,CDG,285,219,?,?,0,0,...,219,8283,0.44,3.29,4.0,0.0,0.0,4.0,278.0,402.43
4,12343,1,FCO,LHR,395,152,?,?,0,0,...,152,74548,0.74,3.46,2.5,0.0,0.0,2.5,897.0,2296.68


In [2]:
flight_data['order_date'] = pd.to_datetime(flight_data.order_date)
flight_data['deliver_date'] = pd.to_datetime(flight_data['deliver_date'])
flight_data['planned_date'] = pd.to_datetime(flight_data['planned_date'])
flight_data.set_index('order_date', inplace=True) # after setting order_date as an index, it will not be in the column
df = flight_data.loc['2019-3-21':'2019-3-28']  # order date in the range

In [3]:
df.reset_index(level=0, inplace=True) # need to reset the index so that the order_date will be back in column
df.sort_values('order_date', inplace=True, ascending=False)
df.reset_index(level=0, inplace=True) # need to reset the index AGAIN so that the orderrange list would be the same order

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
import datetime
from datetime import timedelta
df['planned_date+1']=df['planned_date']+datetime.timedelta(days = 1) 
df['ID']=df.index+0.3
df['ID1']=df.index+0.7
df['from_to'] = df['origin'] + ' - '+ df['desti']

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [5]:
df.loc[:,'color'] = np.where(df['planned_date'] > df['deliver_date'], 'blue', 'red')

/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
from bokeh.io import show, output_file, output_notebook
from bokeh.models import ColumnDataSource, Range1d, SingleIntervalTicker, LinearAxis
from bokeh.plotting import figure
from bokeh.models.tools import HoverTool

# output_file("bar_intervals.html")
output_notebook()

source = ColumnDataSource(df)

# daterange = pd.date_range('2019-3-21', '2019-3-28').strftime("%Y-%m-%d")
# daterange.tolist()

orderrange = df['i1_legid'].astype(str) # i1_legid is unique, so do not need .unique()
orderrange.tolist()

# whatever object you pass into x_range, it must be iterable, containing string type elements only.
# p = figure(y_range=orderrange, x_range=daterange, 
#            plot_width=800, plot_height=3000,toolbar_location=None,
#            title="Delivery Timeline", x_axis_location='above')
p = figure(y_range=orderrange, x_range=Range1d(df.order_date.min(),df.order_date.max()+datetime.timedelta(days=1)), x_axis_type='datetime',
           plot_width=1000, plot_height=3000,toolbar_location=None,
           title="Delivery Timeline", x_axis_location='above')
#p.hbar(y='i1_legid', left='order_date', right='deliver_date', height=0.4, source=source)
p.quad(left='order_date', right='deliver_date', bottom='ID', top='ID1',source=source, color='color')
p.quad(left='planned_date', right='planned_date', bottom='ID', top='ID1',source=source, color='black')

p.ygrid.grid_line_color = None
p.xaxis.axis_label = None
p.outline_line_color = None

hover = HoverTool()
hover.tooltips = [
            ('Order number','#@i1_legid'),
            ('From to', '@from_to'),
            ('Legs', '@i1_hops'),
            ('Expected delivery time', '@deliver_date{%F}'),
           ]

hover.mode = 'mouse'
hover.formatters={'deliver_date': 'datetime'}

# Add the HoverTool to the figure
# fig.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))
p.add_tools(hover)

show(p)

Loading BokehJS ...